In [11]:


# 

# #df.Drug.unique()
# for i in ['Ascites','Hepatomegaly','Spiders','Edema', 'Drug']:
#     print(df[i].unique())

['N' 'Y']
['Y' 'N']
['N' 'Y']
['N' 'Y' 'S']
['D-penicillamine' 'Placebo']


In [17]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

nltk.download('punkt')
nltk.download('punkt_tab')

df = pd.read_csv('liver/train.csv')

df.loc[df['Sex'] == 'F', 'Sex'] = 'Female'
df.loc[df['Sex'] == 'M', 'Sex'] = 'Male'
df.loc[df['Ascites'] == 'N', 'Ascites'] = 'No'
df.loc[df['Ascites'] == 'Y', 'Ascites'] = 'Yes'
df.loc[df['Hepatomegaly'] == 'N', 'Hepatomegaly'] = 'No'
df.loc[df['Hepatomegaly'] == 'Y', 'Hepatomegaly'] = 'Yes'
df.loc[df['Edema'] == 'N', 'Edema'] = 'No'
df.loc[df['Edema'] == 'Y', 'Edema'] = 'Yes'
df.loc[df['Spiders'] == 'N', 'Spiders'] = 'No'
df.loc[df['Spiders'] == 'Y', 'Spiders'] = 'Yes'

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Professional\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Professional\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [18]:
from nltk.tokenize import word_tokenize

# df['Combined_1'] = df[['Drug', 'Sex']].agg(' '.join, axis=1)
# df['Combined_2'] = df[['Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Status']].agg(' '.join, axis=1)
# df['Tokenized_2'] = df['Combined_2'].apply(lambda x: ' '.join(word_tokenize(x)))
# df['Tokenized_1'] = df['Combined_1'].apply(lambda x: ' '.join(word_tokenize(x)))

df['Combined'] = df[['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']].agg(' '.join, axis=1)
df['Tokenized'] = df['Combined'].apply(lambda x: ' '.join(word_tokenize(x)))
#print(df.Tokenized)
print(df.Tokenized)

0         D-penicillamine Male No No No No
1               Placebo Female No No No No
2            Placebo Female No Yes Yes Yes
3               Placebo Female No No No No
4              Placebo Female No Yes No No
                       ...                
7900    D-penicillamine Female No No No No
7901           Placebo Female No Yes No No
7902    D-penicillamine Female No No Yes S
7903     D-penicillamine Male No Yes No No
7904    D-penicillamine Female No No No No
Name: Tokenized, Length: 7905, dtype: object


In [19]:
# tfidf_1 = TfidfVectorizer()
# tfidf_features_1 = tfidf_1.fit_transform(df['Tokenized_1'])

# tfidf_df = pd.DataFrame(tfidf_features_1.toarray(), columns=tfidf_1.get_feature_names_out())
# #--------------------------
# tfidf_2 = TfidfVectorizer(lowercase=False, stop_words=None)
# tfidf_features_2 = tfidf_2.fit_transform(df['Tokenized_2'])

# tfidf_df_2 = pd.DataFrame(tfidf_features_2.toarray(), columns=tfidf_2.get_feature_names_out())
#---------------------------
#--------------------------
tfidf = TfidfVectorizer(lowercase=False, stop_words=None)
tfidf_features = tfidf.fit_transform(df['Tokenized'])

tfidf_df = pd.DataFrame(tfidf_features.toarray(), columns=tfidf.get_feature_names_out())
#---------------------------
# 4. Объединение с числовыми признаками
numeric_features = df[['N_Days', 'Age', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 
                         'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin', 'Stage']]
final_data = pd.concat([numeric_features.reset_index(drop=True), tfidf_df.reset_index(drop=True)], axis=1)

label_encoder = LabelEncoder()
df.Status = label_encoder.fit_transform(df.Status)

print(tfidf_df)

        Female      Male        No   Placebo       Yes  penicillamine
0     0.000000  0.633304  0.714250  0.000000  0.000000       0.297948
1     0.236035  0.000000  0.899095  0.368667  0.000000       0.000000
2     0.205352  0.000000  0.195555  0.320743  0.903722       0.000000
3     0.236035  0.000000  0.899095  0.368667  0.000000       0.000000
4     0.269648  0.000000  0.770350  0.421168  0.395560       0.000000
...        ...       ...       ...       ...       ...            ...
7900  0.235476  0.000000  0.896967  0.000000  0.000000       0.374168
7901  0.269648  0.000000  0.770350  0.421168  0.395560       0.000000
7902  0.327838  0.000000  0.624394  0.000000  0.480921       0.520930
7903  0.000000  0.685919  0.580193  0.000000  0.297918       0.322702
7904  0.235476  0.000000  0.896967  0.000000  0.000000       0.374168

[7905 rows x 6 columns]


In [20]:
X = final_data
y = df['Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
#print(X_test)
accuracy = model.score(X_test, y_test)

y_pred = model.predict(X_test)

print(accuracy)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
confusion_selected1 = confusion_matrix(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:\n', report)

0.8285895003162556
Accuracy: 0.8285895003162556
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.93      0.88       966
           1       0.67      0.12      0.20        52
           2       0.84      0.72      0.77       563

    accuracy                           0.83      1581
   macro avg       0.78      0.59      0.62      1581
weighted avg       0.82      0.83      0.82      1581

